In [2]:
!pip3 install transformers
!pip3 install pillow

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from matrixvit import MatrixViTModel


In [5]:
OUTPUT_DIMENSIONS = (32,32)
BOUNDS = None

In [6]:
model = MatrixViTModel(OUTPUT_DIMENSIONS)

no bounds


In [7]:
import torch
import math
from torch.utils.data import Dataset, DataLoader, random_split
from dummyset import Dummyset, show_predictions

def train(num_epochs, lr=5e-5, batch_size = 32):

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  loss_function = torch.nn.MSELoss()

  dataset = Dummyset("./cifar-set", matrix_dimensions=OUTPUT_DIMENSIONS, size = 10000)
  data_size = len(dataset)
  ptrain = 0.8
  ntrain = math.ceil(data_size * ptrain)
  ntest = data_size - ntrain

  train_set, test_set = random_split(dataset, (ntrain, ntest))

  train_loader = DataLoader(train_set, batch_size)
  test_loader = DataLoader(test_set, batch_size)
  
  model.to(device)

  print('begin training')
  for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for images, matrices in train_loader:
        images, matrices = images.to(device), matrices.to(device)

        outputs = model.forward(images)
        loss = loss_function(outputs, matrices)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Train Loss: {total_loss / len(train_loader)}')

    # Testing loop
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for images, matrices in test_loader:
            images, matrices = images.to(device), matrices.to(device)
            outputs = model.forward(images)
            loss = loss_function(outputs, matrices)
            total_loss += loss.item()
    print(f'Epoch {epoch+1}, Test Loss: {total_loss / len(test_loader)}')
    show_predictions(test_set, dataset.matrix_transform, model)


In [8]:
train(10)

initing
Files already downloaded and verified
data loaded
begin training
